In [1]:
import os
import re
import json
import time
import logging
import threading
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import requests

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('ticket_analyzer.log')
    ]
)
logger = logging.getLogger(__name__)

class TicketAnalyzer:
    def __init__(self, root):
        self.root = root
        self.root.title("Analizador de Tickets IT")
        self.root.geometry("600x400")
        self.root.resizable(False, False)
        
        # Variables
        self.file_path = tk.StringVar()
        self.processing = False
        self.cancel_requested = False
        self.required_columns = [
            'Number', 'Priority', 'State', 'Assigned to', 'Short description',
            'Task type', 'Subcategory', 'Closed', 'Created', 'Assignment group',
            'Work notes list', 'Work notes', 'Description'
        ]
        
        # Cache para respuestas LLM
        self.mistral_cache = {}
        self.gemma3_cache = {}
        
        self.create_widgets()
        
    def create_widgets(self):
        # Frame superior - Selección de archivo
        top_frame = ttk.Frame(self.root, padding="10")
        top_frame.pack(fill=tk.X)
        
        ttk.Label(top_frame, text="Seleccionar archivo Excel:").pack(side=tk.LEFT, padx=(0, 5))
        ttk.Entry(top_frame, textvariable=self.file_path, width=40, state="readonly").pack(side=tk.LEFT, padx=(0, 5))
        ttk.Button(top_frame, text="🖿", command=self.browse_file, width=3).pack(side=tk.LEFT)
        
        # Frame central - Logs y progreso
        center_frame = ttk.Frame(self.root, padding="10")
        center_frame.pack(fill=tk.BOTH, expand=True)
        
        # Text area para logs con scrollbar
        self.log_text = tk.Text(center_frame, height=15, wrap=tk.WORD)
        scrollbar = ttk.Scrollbar(center_frame, command=self.log_text.yview)
        self.log_text.configure(yscrollcommand=scrollbar.set)
        self.log_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Frame para la barra de progreso
        progress_frame = ttk.Frame(self.root, padding="10")
        progress_frame.pack(fill=tk.X)
        
        self.progress_label = ttk.Label(progress_frame, text="Progreso: 0%")
        self.progress_label.pack(side=tk.TOP, anchor=tk.W)
        
        self.progress_bar = ttk.Progressbar(progress_frame, orient=tk.HORIZONTAL, length=580, mode='determinate')
        self.progress_bar.pack(fill=tk.X)
        
        # Frame inferior - Botones de acción
        bottom_frame = ttk.Frame(self.root, padding="10")
        bottom_frame.pack(fill=tk.X)
        
        self.process_button = ttk.Button(bottom_frame, text="▶ Ejecutar Proceso", command=self.start_processing)
        self.process_button.pack(side=tk.LEFT, padx=(0, 5))
        
        self.cancel_button = ttk.Button(bottom_frame, text="⏹ Cancelar", command=self.cancel_processing, state=tk.DISABLED)
        self.cancel_button.pack(side=tk.LEFT)
        
        # Botón para probar un solo ticket
        self.test_button = ttk.Button(bottom_frame, text="🧪 Probar (1 ticket)", command=self.test_single_ticket)
        self.test_button.pack(side=tk.RIGHT)
        
    def browse_file(self):
        file_path = filedialog.askopenfilename(
            title="Seleccionar archivo Excel",
            filetypes=[("Archivos Excel", "*.xlsx")]
        )
        if file_path:
            self.file_path.set(file_path)
            self.log_message(f"Archivo seleccionado: {file_path}")
    
    def log_message(self, message):
        timestamp = datetime.now().strftime("%H:%M:%S")
        self.log_text.insert(tk.END, f"[{timestamp}] {message}\n")
        self.log_text.see(tk.END)
        logger.info(message)
    
    def start_processing(self):
        if not self.file_path.get():
            messagebox.showerror("Error", "Por favor seleccione un archivo Excel primero.")
            return
        
        if self.processing:
            return
            
        self.processing = True
        self.cancel_requested = False
        self.process_button.config(state=tk.DISABLED)
        self.test_button.config(state=tk.DISABLED)
        self.cancel_button.config(state=tk.NORMAL)
        
        # Iniciar procesamiento en un hilo separado
        processing_thread = threading.Thread(target=self.process_file)
        processing_thread.daemon = True
        processing_thread.start()
    
    def test_single_ticket(self):
        """Procesa solo el primer ticket para pruebas"""
        if not self.file_path.get():
            messagebox.showerror("Error", "Por favor seleccione un archivo Excel primero.")
            return
        
        if self.processing:
            return
            
        self.processing = True
        self.cancel_requested = False
        self.process_button.config(state=tk.DISABLED)
        self.test_button.config(state=tk.DISABLED)
        self.cancel_button.config(state=tk.NORMAL)
        
        # Iniciar procesamiento en un hilo separado
        test_thread = threading.Thread(target=self.process_test_ticket)
        test_thread.daemon = True
        test_thread.start()
    
    def process_test_ticket(self):
        """Procesa solo el primer ticket para pruebas"""
        try:
            # Verificar conexión con Ollama
            if not self.check_ollama_connection():
                self.log_message("ERROR: No se pudo conectar con el servidor Ollama. Verifique que está ejecutándose.")
                self.reset_ui()
                return

            # Cargar archivo Excel
            self.log_message("Cargando archivo Excel...")
            df = pd.read_excel(self.file_path.get())
            
            # Validar columnas requeridas
            missing_columns = [col for col in self.required_columns if col not in df.columns]
            if missing_columns:
                self.log_message(f"ERROR: Faltan columnas requeridas: {', '.join(missing_columns)}")
                self.reset_ui()
                return
                
            # Tomar solo el primer ticket
            if len(df) > 0:
                row = df.iloc[0]
                ticket_num = row['Number']
                self.log_message(f"PRUEBA: Procesando ticket #{ticket_num}")
                
                # Limpiar texto para el análisis
                try:
                    short_desc = str(row['Short description']) if not pd.isna(row['Short description']) else ""
                    work_notes = str(row['Work notes']) if not pd.isna(row['Work notes']) else ""
                    description = str(row['Description']) if not pd.isna(row['Description']) else ""
                    
                    # Imprimir información del texto
                    self.log_message(f"Longitud de Short description: {len(short_desc)} caracteres")
                    self.log_message(f"Longitud de Work notes: {len(work_notes)} caracteres")
                    self.log_message(f"Longitud de Description: {len(description)} caracteres")
                    
                    # Combinar texto y limpiarlo
                    combined_text = f"Ticket: {ticket_num}\nDescripción Corta: {short_desc}\nNotas de Trabajo: {work_notes}\nDescripción: {description}"
                    cleaned_text = self.clean_text(combined_text)
                    self.log_message(f"Longitud de texto combinado y limpio: {len(cleaned_text)} caracteres")
                    
                    # Clasificar con Mistral
                    self.log_message("Probando modelo Mistral...")
                    mistral_result = self.process_with_mistral(cleaned_text)
                    self.log_message("✓ Respuesta de Mistral recibida correctamente")
                    self.log_message(f"Longitud de respuesta Mistral: {len(mistral_result)} caracteres")
                    
                    # Resumir con Gemma3
                    self.log_message("Probando modelo Gemma3...")
                    gemma3_result = self.process_with_gemma3(cleaned_text)
                    self.log_message("✓ Respuesta de Gemma3 recibida correctamente")
                    self.log_message(f"Longitud de respuesta Gemma3: {len(gemma3_result)} caracteres")
                    
                    self.log_message("PRUEBA COMPLETADA EXITOSAMENTE ✓")
                    
                except Exception as e:
                    self.log_message(f"ERROR en prueba: {str(e)}")
            else:
                self.log_message("ERROR: El archivo Excel no contiene tickets.")
                
        except Exception as e:
            self.log_message(f"ERROR GENERAL: {str(e)}")
        
        finally:
            self.reset_ui()
    
    def cancel_processing(self):
        if self.processing:
            self.cancel_requested = True
            self.log_message("Cancelación solicitada. Espere a que finalice el procesamiento actual...")
            self.cancel_button.config(state=tk.DISABLED)
    
    def process_file(self):
        start_time = time.time()
        self.log_message("Iniciando procesamiento...")
        
        try:
            # Verificar conexión con Ollama
            if not self.check_ollama_connection():
                self.log_message("ERROR: No se pudo conectar con el servidor Ollama. Verifique que está ejecutándose.")
                self.reset_ui()
                return
                
            # Cargar archivo Excel
            self.log_message("Cargando archivo Excel...")
            df = pd.read_excel(self.file_path.get())
            
            # Validar columnas requeridas
            missing_columns = [col for col in self.required_columns if col not in df.columns]
            if missing_columns:
                self.log_message(f"ERROR: Faltan columnas requeridas: {', '.join(missing_columns)}")
                self.reset_ui()
                return
            
            # Preparar para el procesamiento
            total_tickets = len(df)
            self.log_message(f"Total de tickets a procesar: {total_tickets}")
            
            # Crear nuevas columnas para resultados
            df['DescripcionMistral'] = ""
            df['ResumenGemma3'] = ""
            
            # Procesar tickets
            errors_mistral = 0
            errors_gemma3 = 0
            processed_count = 0
            
            for index, row in df.iterrows():
                if self.cancel_requested:
                    self.log_message("Proceso cancelado por el usuario.")
                    break
                
                ticket_num = row['Number']
                self.log_message(f"Procesando ticket #{ticket_num} ({index + 1}/{total_tickets})...")
                
                # Limpiar texto para el análisis
                try:
                    short_desc = str(row['Short description']) if not pd.isna(row['Short description']) else ""
                    work_notes = str(row['Work notes']) if not pd.isna(row['Work notes']) else ""
                    description = str(row['Description']) if not pd.isna(row['Description']) else ""
                    
                    # Combinar texto y limpiarlo
                    combined_text = f"Ticket: {ticket_num}\nDescripción Corta: {short_desc}\nNotas de Trabajo: {work_notes}\nDescripción: {description}"
                    cleaned_text = self.clean_text(combined_text)
                    
                    # Clasificar con Mistral
                    mistral_start = time.time()
                    mistral_result = self.process_with_mistral(cleaned_text)
                    mistral_time = time.time() - mistral_start
                    df.at[index, 'DescripcionMistral'] = mistral_result
                    self.log_message(f"Mistral procesado en {mistral_time:.2f} segundos")
                    
                    # Para evitar sobrecarga, esperar un momento
                    time.sleep(1)
                    
                    # Resumir con Gemma3
                    gemma_start = time.time()
                    gemma3_result = self.process_with_gemma3(cleaned_text)
                    gemma_time = time.time() - gemma_start
                    df.at[index, 'ResumenGemma3'] = gemma3_result
                    self.log_message(f"Gemma3 procesado en {gemma_time:.2f} segundos")
                    
                    # Guardar progresivamente para no perder datos
                    if index % 5 == 0 and index > 0:
                        self.save_progress(df)
                    
                except Exception as e:
                    self.log_message(f"Error procesando ticket #{ticket_num}: {str(e)}")
                    if "mistral" in str(e).lower():
                        errors_mistral += 1
                    if "gemma" in str(e).lower():
                        errors_gemma3 += 1
                
                # Actualizar progreso cada ticket
                processed_count += 1
                progress = int((processed_count / total_tickets) * 100)
                self.update_progress(progress)
            
            # Guardar resultados finales
            if not self.cancel_requested:
                self.save_results(df)
                
            # Estadísticas finales
            end_time = time.time()
            total_time = end_time - start_time
            self.log_message(f"\nEstadísticas finales:")
            self.log_message(f"Tiempo total: {total_time:.2f} segundos")
            self.log_message(f"Tickets procesados: {processed_count}/{total_tickets}")
            self.log_message(f"Errores con Mistral: {errors_mistral}")
            self.log_message(f"Errores con Gemma3: {errors_gemma3}")
                
        except Exception as e:
            self.log_message(f"ERROR GENERAL: {str(e)}")
        
        finally:
            self.reset_ui()
    
    def check_ollama_connection(self):
        try:
            response = requests.get("http://localhost:11434/api/tags", timeout=5)
            if response.status_code == 200:
                models = response.json().get("models", [])
                mistral_found = any("mistral" in model["name"].lower() for model in models)
                gemma_found = any("gemma" in model["name"].lower() for model in models)
                
                if not mistral_found:
                    self.log_message("ADVERTENCIA: Modelo 'mistral' no encontrado específicamente en Ollama")
                if not gemma_found:
                    self.log_message("ADVERTENCIA: Modelo 'gemma3' no encontrado específicamente en Ollama")
                
                # Listar modelos disponibles
                model_names = [model["name"] for model in models]
                self.log_message(f"Modelos disponibles en Ollama: {', '.join(model_names)}")
                
                return True
            return False
        except Exception as e:
            self.log_message(f"Error al verificar conexión con Ollama: {str(e)}")
            return False
    
    def clean_text(self, text):
        """Limpia el texto de patrones y HTML"""
        # Eliminar patrones [ARGONAUTA] y [INC...]
        text = re.sub(r'\[ARGONAUTA\]|\[INC[^\]]*\]', '', text)
        
        # Remover HTML
        try:
            soup = BeautifulSoup(text, 'html.parser')
            text = soup.get_text(separator=' ')
        except Exception as e:
            self.log_message(f"Advertencia al limpiar HTML: {str(e)}")
            pass
        
        # Eliminar espacios extra y normalizar
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    def prepare_text_for_llm(self, text, max_length=3500):
        """Prepara el texto para enviar al LLM, recortando si es necesario"""
        if len(text) <= max_length:
            return text
        
        # Estrategia de recorte inteligente
        # Mantener el inicio y el final que suelen contener info importante
        start_portion = max_length // 3 * 2  # 2/3 del inicio
        end_portion = max_length // 3        # 1/3 del final
        middle_indicator = "... [contenido omitido para optimizar procesamiento] ..."
        
        result = text[:start_portion] + middle_indicator + text[-end_portion:]
        self.log_message(f"Texto reducido de {len(text)} a {len(result)} caracteres")
        
        return result
    
    def process_with_mistral(self, text):
        """Procesa el texto con el modelo Mistral"""
        # Verificar cache
        cache_key = hash(text)
        if cache_key in self.mistral_cache:
            self.log_message("Usando resultado de cache para Mistral")
            return self.mistral_cache[cache_key]
        
        # Preparar texto optimizado
        prepared_text = self.prepare_text_for_llm(text)
        
        prompt = f"""Clasifica este ticket de IT según categorías técnicas:
Descripción: {prepared_text}
Considerar: Lo que se encuentra en la columna Short description, Work notes y Description, para hacer un analisis del ticket y poder clasificar el ticket
Formato: JSON con clasificación y razonamiento"""
        
        try:
            response = self.call_ollama_api("mistral", prompt)
            
            # Intentar extraer JSON de la respuesta
            json_result = self.extract_json(response)
            if json_result:
                result_str = json.dumps(json_result, ensure_ascii=False)
                # Guardar en cache
                self.mistral_cache[cache_key] = result_str
                return result_str
            else:
                formatted_response = self.format_response(response)
                # Guardar en cache
                self.mistral_cache[cache_key] = formatted_response
                return formatted_response
                
        except Exception as e:
            raise Exception(f"Error con Mistral: {str(e)}")
    
    def process_with_gemma3(self, text):
        """Procesa el texto con el modelo Gemma3"""
        # Verificar cache
        cache_key = hash(text)
        if cache_key in self.gemma3_cache:
            self.log_message("Usando resultado de cache para Gemma3")
            return self.gemma3_cache[cache_key]
        
        # Preparar texto optimizado
        prepared_text = self.prepare_text_for_llm(text, max_length=3000)
        
        prompt = f"""Resume este ticket para análisis de tendencias:
{prepared_text}
Máximo 3 puntos clave técnicos
Identificar posible causa raíz
Formato: Viñetas concisas"""
        
        try:
            response = self.call_ollama_api("gemma3", prompt)
            formatted_response = self.format_response(response)
            
            # Guardar en cache
            self.gemma3_cache[cache_key] = formatted_response
            return formatted_response
            
        except Exception as e:
            raise Exception(f"Error con Gemma3: {str(e)}")
    
    def call_ollama_api(self, model, prompt, max_retries=3):
        """Llama a la API de Ollama con reintentos y timeout extendido"""
        url = "http://localhost:11434/api/generate"
        headers = {"Content-Type": "application/json"}
        
        # Truncar el prompt si es demasiado largo
        if len(prompt) > 4000:
            self.log_message(f"Prompt truncado de {len(prompt)} a 4000 caracteres")
            prompt = prompt[:4000] + "... [contenido truncado]"
        
        # Si el modelo es gemma3 pero no está disponible, intentar con gemma:2b
        if model == "gemma3":
            model_alternatives = ["gemma3", "gemma:2b", "gemma", "gemma:7b"]
        else:
            model_alternatives = [model, "mistral:7b", "mistral", "llama3"]
        
        data = {
            "model": model,
            "prompt": prompt,
            "stream": False
        }
        
        retry_count = 0
        timeout_value = 180  # Aumentado a 3 minutos
        
        while retry_count < max_retries:
            try:
                # Intentar con modelo alternativo si es necesario
                if retry_count > 0 and retry_count < len(model_alternatives):
                    current_model = model_alternatives[retry_count]
                    data["model"] = current_model
                    self.log_message(f"Intentando con modelo alternativo: {current_model}")
                
                self.log_message(f"Enviando solicitud a {data['model']} (timeout: {timeout_value}s)...")
                response = requests.post(url, headers=headers, json=data, timeout=timeout_value)
                
                if response.status_code == 200:
                    return response.json().get("response", "")
                else:
                    retry_count += 1
                    wait_time = 2 ** retry_count  # Backoff exponencial
                    self.log_message(f"Error en API de Ollama (código {response.status_code}). Reintentando en {wait_time} segundos...")
                    time.sleep(wait_time)
                    
            except requests.exceptions.Timeout:
                # Específicamente para timeouts, aumentamos el timeout para el siguiente intento
                retry_count += 1
                timeout_value += 120  # Incrementar en 2 minutos cada vez
                wait_time = 2 ** retry_count
                self.log_message(f"Timeout en la conexión. Aumentando timeout a {timeout_value}s. Reintentando en {wait_time} segundos...")
                time.sleep(wait_time)
                
            except Exception as e:
                retry_count += 1
                wait_time = 2 ** retry_count
                self.log_message(f"Error de conexión: {str(e)}. Reintentando en {wait_time} segundos...")
                time.sleep(wait_time)
        
        raise Exception(f"No se pudo obtener respuesta después de {max_retries} intentos")
    
    def extract_json(self, text):
        """Intenta extraer un objeto JSON de la respuesta"""
        try:
            # Buscar texto entre {} con regex
            json_match = re.search(r'\{.+\}', text, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                return json.loads(json_str)
            
            # Si no funciona, intentar con toda la respuesta
            return json.loads(text)
        except:
            return None
    
    def format_response(self, text):
        """Formatea la respuesta para guardarla en Excel"""
        # Limitar a 32,767 caracteres (límite de celdas Excel)
        if len(text) > 32000:
            text = text[:32000] + "... (truncado)"
        return text
    
    def save_progress(self, df):
        """Guarda un archivo temporal de progreso"""
        try:
            temp_file = f"temp_progress_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
            df.to_excel(temp_file, index=False)
            self.log_message(f"Progreso guardado en archivo temporal: {temp_file}")
        except Exception as e:
            self.log_message(f"Error al guardar progreso temporal: {str(e)}")
    
    def save_results(self, df):
        """Guarda los resultados en un nuevo archivo Excel"""
        try:
            # Solicitar ubicación para guardar
            output_path = filedialog.asksaveasfilename(
                title="Guardar resultados",
                defaultextension=".xlsx",
                filetypes=[("Archivos Excel", "*.xlsx")]
            )
            
            if not output_path:
                self.log_message("Guardado cancelado por el usuario.")
                return
                
            # Guardar el DataFrame a Excel
            df.to_excel(output_path, index=False)
            self.log_message(f"Resultados guardados exitosamente en: {output_path}")
            
        except Exception as e:
            self.log_message(f"Error al guardar resultados: {str(e)}")
    
    def update_progress(self, value):
        """Actualiza la barra de progreso"""
        self.root.after(0, lambda: self._update_progress_ui(value))
    
    def _update_progress_ui(self, value):
        self.progress_bar["value"] = value
        self.progress_label.config(text=f"Progreso: {value}%")
        
    def reset_ui(self):
        """Restablece la interfaz después del procesamiento"""
        self.root.after(0, lambda: self._reset_ui_internal())
    
    def _reset_ui_internal(self):
        self.processing = False
        self.process_button.config(state=tk.NORMAL)
        self.test_button.config(state=tk.NORMAL)
        self.cancel_button.config(state=tk.DISABLED)

if __name__ == "__main__":
    root = tk.Tk()
    app = TicketAnalyzer(root)
    root.mainloop()

2025-04-18 20:32:01,246 - INFO - Archivo seleccionado: /home/danny/Downloads/Casos2.xlsx
2025-04-18 20:32:03,151 - INFO - Iniciando procesamiento...
2025-04-18 20:32:03,156 - INFO - Modelos disponibles en Ollama: gemma3:latest, mistral:latest, deepseek-r1:8b, gemma3:12b, llama3.2:latest, deepseek-r1:latest, deepseek-r1:14b
2025-04-18 20:32:03,158 - INFO - Cargando archivo Excel...
2025-04-18 20:32:03,328 - INFO - Total de tickets a procesar: 9
2025-04-18 20:32:03,330 - INFO - Procesando ticket #SCTASK0000233249 (1/9)...
2025-04-18 20:32:03,331 - INFO - Enviando solicitud a mistral (timeout: 180s)...
2025-04-18 20:34:11,364 - INFO - Mistral procesado en 128.03 segundos
2025-04-18 20:34:12,365 - INFO - Enviando solicitud a gemma3 (timeout: 180s)...
2025-04-18 20:35:21,919 - INFO - Gemma3 procesado en 69.55 segundos
2025-04-18 20:35:21,921 - INFO - Procesando ticket #SCTASK0000244189 (2/9)...
2025-04-18 20:35:21,924 - INFO - Texto reducido de 12065 a 3554 caracteres
2025-04-18 20:35:21,92